<a href="https://colab.research.google.com/github/amannarsaria4/fast.ai/blob/master/Lesson%202/(MyCode)lesson2_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.vision import *

###Create Directory, Upload URL, Download Images

In [2]:
folder = 'black'
file = 'urls_black.txt'

In [3]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(exist_ok = True, parents = True)

In [ ]:
download_images(dest/file,dest,max_pics = 200)

In [6]:
folder = 'grizzly'
file = 'urls_grizzly.txt'

In [7]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(exist_ok = True, parents = True)

In [ ]:
download_images(dest/file,dest,max_pics = 200)

In [9]:
folder = 'teddy'
file = 'urls_teddy.txt'

In [25]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(parents = True, exist_ok = True)

In [ ]:
download_images(dest/file, dest, max_pics = 200)

In [ ]:
classes = ['teddy','grizzly','black']
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_size=500)

###Show Data

In [27]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [ ]:
data.show_batch()

In [ ]:
data.c,data.classes, len(data.train_ds), len(data.valid_ds)

###Learn Model(Resnet 34) and Fine Tune

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics = error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [33]:
learn.save('stage-1')

In [34]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [39]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(2, max_lr = slice(1e-5,1e-4))

In [43]:
learn.save('stage-2')

###Interpret

In [ ]:
interpret = ClassificationInterpretation.from_learner(learn)

In [ ]:
interpret.plot_confusion_matrix()

###Clean Data

In [45]:
from fastai.widgets import *


In [ ]:
data = ImageList.from_folder(path).split_none().label_from_folder().transform(get_transforms(),size = 224).databunch()

In [ ]:
learn_clean = cnn_learner(data, models.resnet34, metrics=error_rate)
learn_clean.laod('stage-2')

In [ ]:
ds, idxs = DatasetFormatter().from_top_losses(learn_clean)

In [ ]:
ImageCleaner(ds, idxs, path)

In [ ]:
data = ImageList.from_csv(path, 'cleaned.csv', folder = '.').split_none().label_from_df().transform(get_transforms(), size = 224).databunch()

In [ ]:
learn_clean = cnn_learner(data, models.resnet34, metrics=error_rate)
learn_clean.laod('stage-2')

In [ ]:
ds, idxs = DatasetFormatter().from_top_losses(learn_clean)

In [ ]:
ImageCleaner(ds, idxs, path)

###Reatrain on clean data set

In [ ]:
data = ImageList.from_csv(path, 'cleaned.csv',folder = '.').split_none().label_from_df().transform(get_transform(), size = 224).databunch()

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics = error_rate)

In [ ]:
learn.fit_one_cycle(5)
learn.save('stage-3')

###Production

In [ ]:
learn.export()

In [ ]:
img = open_image(path/'bears'/'0000034.jpg')

In [ ]:
learn = load_learner(path)

In [ ]:
pred_class, pred_idx, prob = learn.predict(img)

In [ ]:
pred_class